# “Words from words” game

## The “Regular Expression”

### RegEx Maker

In [1]:
from itertools import groupby

def make_wfw_regex(word):
    return "^{}[{}]+$".format(
        "".join(
            "(?!(.*{}){{{}}})".format(char, freq) 
            for char, freq in map(
                lambda item: (
                    item[0], 
                    sum(1 for _ in item[1]) + 1
                ), 
                groupby(sorted(word))
            )
        ),
        "".join(set(word))
    )

### Words searcher

In [2]:
import re

def make_words_filter(parent_word):
    pattern = re.compile(make_wfw_regex(parent_word))
    return lambda word: word != parent_word and pattern.match(word)

def find_word_combinations(words_dict, *words):
    return map(
        lambda word: (
            word,
            filter(
                make_words_filter(word), 
                words_dict
            )
        ),
        words
    )

----

## The “Dictionary”

### Dict structure

Format: `csv`/`sql-table`

Fields:
* `index` — int, primary key
* `word` — string, the word
* `solution_count` — int, count of possible solutions based on the same dict

CSV file looks like:

index|word|solution_count
-----|-----|-----
0|абажур|10
1|абаз|1
2|абазин|5
3|абазинка|15
4|абака|0
5|аббат|0
6|аббатиса|6
7|аббатисса|7
8|аббатство|14
9|аббревиатура|50

### Converter

Words list loader:

In [ ]:
import codecs

def load_words(dict_path):
    with codecs.open(dict_path, encoding="utf-8") as fp:
        return fp.read().splitlines()

Dict converter with ansvers counting:

In [ ]:
import pandas as pd

def calc_solutions_count(words_dict):
    return pd.DataFrame(
         [(word, sum(1 for _ in words)) 
          for word, words in find_word_combinations(
              words_dict,
              *words_dict
          )],
        columns=["word", "solution_count"]
    )

Convert words list to csv with wolutions count:

In [ ]:
calc_solutions_count(load_words("rus-nouns.txt")).to_csv("wfw-words.csv")

### Dict manipulations

#### Loading

In [3]:
import pandas as pd

WORDS_DICT_PATH = "wfw-words.csv"

def load_dict(dict_path=WORDS_DICT_PATH):
    df = pd.read_csv(WORDS_DICT_PATH)
    df.set_index(["word"])
    return df

#### Select words

In [4]:
def select_by_solution_range(words_dict, solutions_range):
    df = words_dict[
        words_dict[
            "solution_count"
        ].isin(
            solutions_range
        )
    ].sample(n=1)
    
    return df.values[0][1:]

In [5]:
def has_word(words_dict, *word):
    return words_dict[words_dict["word"].isin(word)].shape[0]

In [6]:
def has_solution(words_dict, word, answer):
    return any(
        any(answer == solution for solution in words)
        for _, words in find_word_combinations(
            words_dict["word"],
            word
        )
    )

----

## Gameplay

### Game levels

In [7]:
def get_levels():
    return zip(
        [10, 20, 35, 50, 100, 5000], 
        ["very easy", "easy", "normal", "hard", "very hard", "insane"]
    )

* Levels:
  ```json
  [(10, 'very easy'),
   (20, 'easy'),
   (35, 'normal'),
   (50, 'hard'),
   (100, 'very hard'),
   (5000, 'insane')]
  ```

### Session

In [8]:
from itertools import islice

def get_level_solutions_range(levels, level_idx):
    return next(islice(
        map(
            lambda item: (item[1][0], item[0][0]),
            zip(levels[1:], levels), 
        ),
        level_idx, 
        level_idx + 1
    ))

In [9]:
def new_session(words_dict, levels, difficult_level):
    return {
       "game": dict(
           zip(
               ["word", "solutions"],
               select_by_solution_range(
                   words_dict, 
                   range(*get_level_solutions_range(levels, 0))
               )
           )
        ),
        "answers": []
    }

* Session:
```json
{'answers': [],
 'game': {
     'solutions': 13, 
     'word': 'выгонщик'
 }}
```

In [10]:
def answer(words_dict, session, answer):
    if not has_word(words_dict, answer):
        return "Unknown word '{}'".format(answer)
    elif answer in session["answers"]:
        return "Word '{}' already solved".format(answer)
    elif not has_solution(words_dict, session["game"]["word"], answer):
        return "Invalid solution '{}'".format(answer)
    else:
        session["answers"].append(answer)

In [11]:
def is_end_game(session):
    return len(session["answers"]) == session["game"]["solutions"]

----

## The “Demo”

In [12]:
words_dict = load_dict()

levels = list(get_levels())

session = new_session(words_dict, levels, 1)

print("Current word:", session["game"]["word"])

for _, words in find_word_combinations(
            words_dict["word"],
            session["game"]["word"]
        ):
    for word in words:
        print("Answering:", word)
        answer(words_dict, session, word)
        print("Is endgame:", is_end_game(session))

Current word: выгонщик
Answering: виг
Is endgame: False
Answering: вино
Is endgame: False
Answering: воин
Is endgame: False
Answering: выгон
Is endgame: False
Answering: гон
Is endgame: False
Answering: гонщик
Is endgame: False
Answering: иго
Is endgame: False
Answering: инок
Is endgame: False
Answering: ион
Is endgame: False
Answering: ков
Is endgame: False
Answering: кон
Is endgame: False
Answering: нок
Is endgame: False
Answering: овин
Is endgame: True
